In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis


In [4]:
class Explanatory_Analysis:
    def __init__(self, base_folder_path):
        self.base_folder_path = base_folder_path
        
    def load_data(self, file_path):
        data = pd.read_csv(file_path)
        data['Day'] = pd.to_datetime(data['Day'])
        data['ret'] = data['close'].pct_change()
        if 'Date' in data.columns:
            data['Date'] = pd.to_datetime(data['Date'])
            data = data[data['Date'].dt.time !=pd.to_datetime('09:30:00').time()]
        return data
    
    def check_interval(self, min_data, threshold = 66):
        daily_counts = min_data.groupby(min_data['Day']).size()
        valid_days = daily_counts[daily_counts > threshold].index
        
        return valid_days
    
    def filter_daily_data(self, daily_data, valid_days):
        daily_data = daily_data.iloc[1:]
        return daily_data[daily_data['Day'].isin(valid_days)]
        
        
    def compute_statistics(self, data):
        return {
            'Obs' : len(data),
            'Mean' : data['ret'].mean(),
            'Std' : data['ret'].std(),
            'Median' : data['ret'].median(),
            'Skewness' : skew(data['ret']),
            'Kurtosis' : kurtosis(data['ret'], fisher=False)
        }
        
    # def plot_data(self,data,ticker):
    #     plt.figure(figsize=(10,6))
    #     plt.plot(data['log_ret'])
    #     plt.title(f'{ticker} Log Returns')
    #     plt.xlabel('Time')
    #     plt.ylabel('Log Returns')
    #     plt.show()
        
        
    def analyse(self):
        results = []
        for folder in os.listdir(self.base_folder_path):
            folder_path = os.path.join(self.base_folder_path, folder)
            if os.path.isdir(folder_path):
                minute_files = [f for f in os.listdir(folder_path) if '_filtered_data' in f]
                daily_files = [f for f in os.listdir(folder_path) if 'daily' in f]
                for min_file, daily_file in zip(minute_files, daily_files):
                    min_data_path = os.path.join(folder_path, min_file)
                    daily_data_path = os.path.join(folder_path, daily_file)
                    
                    min_data = self.load_data(min_data_path)
                    daily_data = self.load_data(daily_data_path)
                    daily_data = daily_data.iloc[1:] 
                    
                    valid_days = self.check_interval(min_data)
                    
                    filtered_daily_data = self.filter_daily_data(daily_data, valid_days)
                    stats = self.compute_statistics(filtered_daily_data)
                    stats['Ticker'] = daily_file.split('_')[0]
                    results.append(stats)
        return pd.DataFrame(results)
            
    # def dataframe_to_latex(self, results_df):
    #     latex_table = results_df.to_latex(index=False, longtable=True,
    #                                   caption="Summary statistics for return and log(RK) across different stocks.",
    #                                   label="tab:stats", 
    #                                   column_format='lcccccc',
    #                                   float_format="{:0.2f}".format)
    #     latex_table = "\\begin{table}[htbp]\n\\centering\n" + \
    #                 latex_table + \
    #                 "\\end{table}"
    
    #     return latex_table


    
        

In [68]:
import os
import pandas as pd
from scipy.stats import skew, kurtosis

class Explanatory_Analysis:
    def __init__(self, base_folder_path):
        self.base_folder_path = base_folder_path
        
    def load_data(self, file_path):
        data = pd.read_csv(file_path)
        return data
    
    def compute_statistics(self, data, column, metrics):
        stats = {}
        if column not in data.columns:
            print(f"Column {column} not found in {file_path}. Available columns: {data.columns}")
            return stats

        if 'Obs' in metrics:
            stats['Obs'] = len(data)
        if 'Mean' in metrics:
            stats['Mean'] = data[column].mean()
        if 'Std' in metrics:
            stats['Std'] = data[column].std()
        if 'Median' in metrics:
            stats['Median'] = data[column].median()
        if 'Skew' in metrics:
            stats['Skew'] = skew(data[column])
        if 'Kurt' in metrics:
            stats['Kurt'] = kurtosis(data[column], fisher=False)
        if 'Q1' in metrics:
            stats['Q1'] = data[column].quantile(0.25)
        if 'Q3' in metrics:
            stats['Q3'] = data[column].quantile(0.75)
        return stats
        
    def analyse(self):
        results = []
        for file in os.listdir(self.base_folder_path):
            file_path = os.path.join(self.base_folder_path, file)
            if os.path.isfile(file_path) and file.endswith('.csv'):
                data = self.load_data(file_path)

                # Multiply returns and RM by 100
                data['returns2'] = data['returns'] * 100
                data['RM2'] = data['RM'] * 100
                
                return_stats = self.compute_statistics(data, 'returns2', ['Obs', 'Mean', 'Std', 'Median', 'Skew', 'Kurt'])
                rm_stats = self.compute_statistics(data, 'RM2', ['Mean', 'Std', 'Q1', 'Median', 'Q3'])
                
                stats = {**return_stats, **rm_stats}
                stats['Ticker'] = file.split('.')[0]  # Remove the .csv extension
                results.append(stats)
                
        result_df = pd.DataFrame(results)
        result_df.set_index('Ticker', inplace=True)
        
        # Sort by Ticker
        result_df.sort_index(inplace=True)
        
        # Rearrange columns to match the desired multi-level structure
        result_df = result_df[['Obs', 'Mean', 'Std', 'Skew', 'Kurt', 'Median', 'Mean', 'Std', 'Q1', 'Median', 'Q3']]
        
        # Rename columns to multi-index
        result_df.columns = pd.MultiIndex.from_tuples([
            ('Return', 'Obs'), ('Return', 'Mean'), ('Return', 'Std'), ('Return', 'Skew'), 
            ('Return', 'Kurt'), ('Return', 'Median'), ('Log(RV)', 'Mean'), ('Log(RV)', 'Std'), 
            ('Log(RV)', 'Q1'), ('Log(RV)', 'Median'), ('Log(RV)', 'Q3')
        ])
        
        result_df.to_csv(os.path.join(self.base_folder_path, 'summary_statistics.csv'))
        return result_df

# Assuming the data files are in '/mnt/data/'
base_folder_path = '/Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2'
output_folder_path = '/Users/raphaelravinet/Code/BSE/Thesis/Results'
# Create an instance of the Explanatory_Analysis class
analysis = Explanatory_Analysis(base_folder_path)

# Run the analysis and get the result DataFrame
result_df = analysis.analyse()

# Generate LaTeX table
latex_table = result_df.to_latex(multicolumn=True, multicolumn_format='c', index=True, float_format="%.2f", 
                                 column_format="l" + "r" * len(result_df.columns), 
                                 caption="Summary statistics for return and log(RK)")

# Save the LaTeX table to a file
with open(os.path.join(output_folder_path, 'summary_statistics_table_latest.tex'), 'w') as f:
    f.write(latex_table)

# Display the LaTeX table
print(latex_table)


\begin{table}
\caption{Summary statistics for return and log(RK)}
\begin{tabular}{lrrrrrrrrrrr}
\toprule
 & \multicolumn{6}{c}{Return} & \multicolumn{5}{c}{Log(RV)} \\
 & Obs & Mean & Std & Skew & Kurt & Median & Mean & Std & Q1 & Median & Q3 \\
Ticker &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
AAPL & 5140 & 0.04 & 0.09 & 0.10 & 8.32 & 0.02 & 0.04 & 0.09 & 0.01 & 0.02 & 0.04 \\
AMGN & 5106 & 0.03 & 0.08 & 0.66 & 11.07 & 0.02 & 0.03 & 0.08 & 0.01 & 0.02 & 0.03 \\
AMZN & 5109 & 0.06 & 0.16 & 0.78 & 17.01 & 0.03 & 0.06 & 0.16 & 0.02 & 0.03 & 0.05 \\
AXP & 5107 & 0.05 & 0.14 & 0.76 & 19.56 & 0.01 & 0.05 & 0.14 & 0.01 & 0.01 & 0.03 \\
BA & 5110 & 0.05 & 0.18 & 0.26 & 19.56 & 0.02 & 0.05 & 0.18 & 0.01 & 0.02 & 0.04 \\
CAT & 5107 & 0.04 & 0.09 & -0.05 & 8.31 & 0.02 & 0.04 & 0.09 & 0.01 & 0.02 & 0.04 \\
CRM & 4897 & 0.07 & 0.12 & 0.65 & 10.91 & 0.04 & 0.07 & 0.12 & 0.02 & 0.04 & 0.07 \\
CSCO & 5107 & 0.04 & 0.09 & -0.14 & 14.03 & 0.02 & 0.04 & 0.09 & 0.01 & 0.02 & 0.03 \\
CVX & 5108 & 0.03 &

In [18]:
a = pd.read_csv('/Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2/AMGN.csv')
a['returns'].mean()

0.03945878524389569

In [19]:
a

,Day,log_ret,returns,RM,log_x,RM_day,RM_week,RM_month,RM_adj,log_x_adj,IV_day,IV_week,IV_month
0,2003-10-10,-0.007090,-0.007064,0.000182,-8.609856,-7.303229,-8.359883,-8.484476,0.000154,-8.777352,2.934920,2.965250,2.991162
1,2003-10-13,0.016217,0.016349,0.000228,-8.384463,-8.609856,-8.234036,-8.457056,0.000193,-8.551959,2.915064,2.956376,2.989644
2,2003-10-14,-0.004927,-0.004915,0.000217,-8.436507,-8.384463,-8.191417,-8.477830,0.000183,-8.604003,2.865054,2.942410,2.990433
3,2003-10-15,-0.022401,-0.022152,0.000152,-8.789856,-8.436507,-8.194117,-8.441639,0.000129,-8.957352,2.854745,2.917227,2.992688
4,2003-10-16,-0.002606,-0.002602,0.000145,-8.835991,-8.789856,-8.183514,-8.400263,0.000123,-9.003486,2.873000,2.892446,2.992382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101,2024-03-22,-0.002820,-0.002816,0.000091,-9.300585,-8.510174,-9.017374,-8.808237,0.000077,-9.468081,2.558777,2.631104,2.646405
5102,2024-03-25,0.016697,0.016837,0.000071,-9.558316,-9.300585,-9.065466,-8.757522,0.000060,-9.725812,2.569554,2.603818,2.642727
5103,2024-03-26,0.003377,0.003383,0.000186,-8.590285,-9.558316,-9.073258,-8.763666,0.000157,-8.757781,2.579459,2.580617,2.641871
5104,2024-03-27,0.015949,0.016077,0.000135,-8.910530,-8.590285,-9.107803,-8.791557,0.000114,-9.078026,2.583243,2.568953,2.642169


In [3]:
base_folder_path = '/Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2'
analysis = Explanatory_Analysis(base_folder_path)
result_df = analysis.analyse()
print(result_df)

Processing file: CSCO.csv
Columns in CSCO.csv: Index(['Day', 'log_ret', 'returns', 'RM', 'log_x', 'RM_day', 'RM_week',
       'RM_month', 'RM_adj', 'log_x_adj', 'IV_day', 'IV_week', 'IV_month'],
      dtype='object')
Processing file: BA.csv
Columns in BA.csv: Index(['Day', 'log_ret', 'returns', 'RM', 'log_x', 'RM_day', 'RM_week',
       'RM_month', 'RM_adj', 'log_x_adj', 'IV_day', 'IV_week', 'IV_month'],
      dtype='object')
Processing file: MRK.csv
Columns in MRK.csv: Index(['Day', 'log_ret', 'returns', 'RM', 'log_x', 'RM_day', 'RM_week',
       'RM_month', 'RM_adj', 'log_x_adj', 'IV_day', 'IV_week', 'IV_month'],
      dtype='object')
Processing file: QQQ.csv
Columns in QQQ.csv: Index(['Day', 'log_ret', 'returns', 'RM', 'log_x', 'RM_day', 'RM_week',
       'RM_month', 'RM_adj', 'log_x_adj', 'IV_day', 'IV_week', 'IV_month'],
      dtype='object')
Processing file: PG.csv
Columns in PG.csv: Index(['Day', 'log_ret', 'returns', 'RM', 'log_x', 'RM_day', 'RM_week',
       'RM_month', 'RM_ad

In [6]:
import pandas as pd

# Load the data
file_path = '/Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2/summary_statistics.csv'
data = pd.read_csv(file_path)

# Define a function to format the table for LaTeX
def generate_latex_table(df):
    columns = [
        "Ticker", "returns_Obs", "returns_Mean", "returns_Std", "returns_Skew", "returns_Kurt", "returns_Median",
        "RM_Mean", "RM_Std", "RM_Q1", "RM_Median", "RM_Q3"
    ]
    
    # Ensure the correct column renaming
    df.columns = [
        "Ticker", "Obs", "Mean", "Std", "Skew", "Kurt", "Med",
        "Mean_logRK", "Std_logRK", "Q1_logRK", "Med_logRK", "Q3_logRK"
    ]
    
    # LaTeX formatting
    latex_table = df.to_latex(index=False, column_format="lcccccccccccc", header=True, float_format="%.2f")
    
    return latex_table

# Generate the LaTeX table
latex_table = generate_latex_table(data)

# Save the LaTeX table to a file
output_path = '//Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2/summary_statistics.tex'
with open(output_path, 'w') as f:
    f.write(latex_table)

output_path


'//Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2/summary_statistics.tex'

In [5]:
analyser = Explanatory_Analysis('/Users/raphaelravinet/Code/BSE/Thesis/data')
analysis_results = analyser.analyse()
latex_table = analyser.dataframe_to_latex(analysis_results)

In [13]:
test = analysis_results[analysis_results['Obs'] > 5000]

In [14]:
print(test['Obs'].min())
print(test['Obs'].max())

5126
5159


In [8]:
analysis_results[analysis_results['Obs'] > 5000]

,Obs,Mean,Std,Median,Skewness,Kurtosis,Ticker
0,5129,0.001154,0.024312,0.000647,0.779542,16.932021,AMZN
1,5127,0.000679,0.019741,0.000705,-0.052879,8.314125,CAT
2,5159,0.001374,0.020808,0.000998,0.097474,8.260797,AAPL
3,5127,0.000181,0.014612,0.000447,-0.327849,11.041736,MMM
4,5127,0.000290,0.012612,0.000380,0.180266,15.569177,WMT
5,5128,0.000470,0.017771,0.000777,0.118549,24.013455,CVX
6,5130,0.000283,0.019920,0.000440,-0.152252,11.219289,INTC
7,5127,0.000521,0.021716,0.000380,0.793792,21.588702,GS
8,5127,0.000615,0.016337,0.000548,-0.000399,13.522239,HD
9,5127,0.000298,0.015833,0.000321,-0.988397,28.137418,MRK


In [22]:
print(latex_table)

\begin{table}[htbp]
\centering
\begin{longtable}{lcccccc}
\caption{Summary statistics for return and log(RK) across different stocks.} \label{tab:stats} \\
\toprule
Obs & Mean & Std & Median & Skewness & Kurtosis & Ticker \\
\midrule
\endfirsthead
\caption[]{Summary statistics for return and log(RK) across different stocks.} \\
\toprule
Obs & Mean & Std & Median & Skewness & Kurtosis & Ticker \\
\midrule
\endhead
\midrule
\multicolumn{7}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
5129 & 0.00 & 0.02 & 0.00 & 0.78 & 16.93 & AMZN \\
5127 & 0.00 & 0.02 & 0.00 & -0.05 & 8.31 & CAT \\
5159 & 0.00 & 0.02 & 0.00 & 0.10 & 8.26 & AAPL \\
5127 & 0.00 & 0.01 & 0.00 & -0.33 & 11.04 & MMM \\
5127 & 0.00 & 0.01 & 0.00 & 0.18 & 15.57 & WMT \\
5128 & 0.00 & 0.02 & 0.00 & 0.12 & 24.01 & CVX \\
5130 & 0.00 & 0.02 & 0.00 & -0.15 & 11.22 & INTC \\
5127 & 0.00 & 0.02 & 0.00 & 0.79 & 21.59 & GS \\
5127 & 0.00 & 0.02 & 0.00 & -0.00 & 13.52 & HD \\
5127 & 0.00 & 0.02 & 0.00 & -0.9

In [35]:
from scipy import stats

def calculate_loss_differential(actual, forecast1, forecast2, crit="MSE", lambda_=0.1):
    if crit == "MSE":
        loss1 = (actual - forecast1) ** 2
        loss2 = (actual - forecast2) ** 2
    elif crit == "MAD":
        loss1 = np.abs(actual - forecast1)
        loss2 = np.abs(actual - forecast2)
    elif crit == "ASYM":
        loss1 = asymmetric_loss(actual - forecast1, lambda_)
        loss2 = asymmetric_loss(actual - forecast2, lambda_)
    else:
        raise ValueError("Criterion must be either 'MSE', 'MAD', or 'ASYM'")
    
    d = loss1 - loss2
    return d



def asymmetric_loss(e, lambda_):
    return np.exp(lambda_ * e) - 1 - lambda_ * e


def bartlett_kernel(j, H):
    return 1 - (j / H)

def long_run_variance(d, H):
    T = len(d)
    # Step 1: Calculate gamma_0 (variance of d)
    gamma_0 = np.var(d, ddof=1)
    
    # Step 2: Calculate the sum of weighted autocovariances
    gamma_sum = 0
    for j in range(1, H + 1):
        weight = bartlett_kernel(j, H)
        # Calculate autocovariance at lag j
        cov = np.cov(d[:-j], d[j:])[0, 1]
        # Apply Bartlett kernel weight
        gamma_sum += weight * cov
    
    # Step 3: Combine gamma_0 and weighted autocovariances to get LRV
    LRV = gamma_0 + 2 * gamma_sum
    
    return LRV


def dm_statistic(mean_d, LRV, T):
    S = mean_d / np.sqrt(LRV / T)
    return S

def dm_test_manual(actual, forecast1, forecast2, h=1, crit="MSE"):
    d = calculate_loss_differential(actual, forecast1, forecast2, crit)
    mean_d, T = np.mean(d), len(d)
    H = int(4 * (T / 100) ** (2/9))  # Optimal bandwidth parameter
    LRV = long_run_variance(d, H)
    S = dm_statistic(mean_d, LRV, T)
    p_value = 2 * (1 - stats.norm.cdf(np.abs(S)))
    return {"DM": S, "p_value": p_value}

# Directory paths
forecast_dir = "/Users/raphaelravinet/Code/BSE/Thesis/Results/Forecast_results_final"
test_data_dir = "/Users/raphaelravinet/Code/BSE/Thesis/Results/DM_TEST"

# List of stocks to analyze
stocks = ["AAPL", "SPY", "IBM", "PG", "WMT", "DIS", "JPM"]

# Models and horizons
models = ["realised_garch_horizon_", "realised_har_garch_horizon_"]
horizons = [1, 5, 10, 20]

# Placeholder for results
results_df = pd.DataFrame()

# Loop through each stock and each model to calculate metrics
for stock in stocks:
    # Load actual data from CSV
    actual_data_path = os.path.join(test_data_dir, f"{stock}_test_data.csv")
    actual_data = pd.read_csv(actual_data_path)
    actual = actual_data['log_x_adj']
    
    for horizon in horizons:
        for i in range(len(models) - 1):
            for j in range(i + 1, len(models)):
                model1_name = f"{models[i]}{horizon}"
                model2_name = f"{models[j]}{horizon}"
                
                forecast_file1 = os.path.join(forecast_dir, f"{stock}.csv", f"{model1_name}.csv")
                forecast_file2 = os.path.join(forecast_dir, f"{stock}.csv", f"{model2_name}.csv")
                
                if os.path.exists(forecast_file1) and os.path.exists(forecast_file2):
                    forecast_data1 = pd.read_csv(forecast_file1)
                    forecast_data2 = pd.read_csv(forecast_file2)
                    
                    forecast1 = forecast_data1['forecast_data']
                    forecast2 = forecast_data2['forecast_data']
                    
                    actual_adjusted = actual[:len(actual) - (horizon - 1)]
                    
                    # Ensure the forecast length matches the adjusted actual length
                    forecast1 = forecast1[:len(actual_adjusted)]
                    forecast2 = forecast2[:len(actual_adjusted)]
                    
                    # Perform DM test manually
                    dm_test_result_mad = dm_test_manual(actual_adjusted, forecast1, forecast2, h=horizon, crit="MAD")
                    dm_test_result_mse = dm_test_manual(actual_adjusted, forecast1, forecast2, h=horizon, crit="MSE")
                    
                    # Create a DataFrame for the current result
                    current_result = pd.DataFrame([{
                        "Stock": stock,
                        "Horizon": horizon,
                        "Model1": model1_name,
                        "Model2": model2_name,
                        "DM_test_statistic_MAD": dm_test_result_mad["DM"],
                        "DM_p_value_MAD": dm_test_result_mad["p_value"],
                        "DM_test_statistic_MSE": dm_test_result_mse["DM"],
                        "DM_p_value_MSE": dm_test_result_mse["p_value"]
                    }])
                    
                    # Concatenate with the results_df DataFrame
                    results_df = pd.concat([results_df, current_result], ignore_index=True)



In [37]:
results_df

,Stock,Horizon,Model1,Model2,DM_test_statistic_MAD,DM_p_value_MAD,DM_test_statistic_MSE,DM_p_value_MSE
0,AAPL,1,realised_garch_horizon_1,realised_har_garch_horizon_1,8.936382,0.000000e+00,7.465740,8.282264e-14
1,AAPL,5,realised_garch_horizon_5,realised_har_garch_horizon_5,22.975156,0.000000e+00,21.610257,0.000000e+00
2,AAPL,10,realised_garch_horizon_10,realised_har_garch_horizon_10,28.573508,0.000000e+00,27.080255,0.000000e+00
3,AAPL,20,realised_garch_horizon_20,realised_har_garch_horizon_20,27.895878,0.000000e+00,27.138034,0.000000e+00
4,SPY,1,realised_garch_horizon_1,realised_har_garch_horizon_1,5.052979,4.349728e-07,5.181915,2.196190e-07
5,SPY,5,realised_garch_horizon_5,realised_har_garch_horizon_5,9.362759,0.000000e+00,9.891687,0.000000e+00
6,SPY,10,realised_garch_horizon_10,realised_har_garch_horizon_10,12.575937,0.000000e+00,13.165348,0.000000e+00
7,SPY,20,realised_garch_horizon_20,realised_har_garch_horizon_20,11.708838,0.000000e+00,11.619116,0.000000e+00
8,IBM,1,realised_garch_horizon_1,realised_har_garch_horizon_1,-0.895736,3.703938e-01,-1.034275,3.010077e-01
9,IBM,5,realised_garch_horizon_5,realised_har_garch_horizon_5,0.377658,7.056843e-01,2.403428,1.624216e-02


In [ ]:
# Directory paths
forecast_dir = "/Users/raphaelravinet/Code/BSE/Thesis/Results/Forecast_results_final"
test_data_dir = "/Users/raphaelravinet/Code/BSE/Thesis/Results/DM_TEST"

In [38]:
import os
import numpy as np
import pandas as pd
from scipy import stats

def asymmetric_loss(e, lambda_):
    return np.exp(lambda_ * e) - 1 - lambda_ * e

def calculate_loss_differential(actual, forecast1, forecast2, crit="MSE", lambda_=0.1):
    if crit == "MSE":
        loss1 = (actual - forecast1) ** 2
        loss2 = (actual - forecast2) ** 2
    elif crit == "MAD":
        loss1 = np.abs(actual - forecast1)
        loss2 = np.abs(actual - forecast2)
    elif crit == "ASYM":
        loss1 = asymmetric_loss(actual - forecast1, lambda_)
        loss2 = asymmetric_loss(actual - forecast2, lambda_)
    else:
        raise ValueError("Criterion must be either 'MSE', 'MAD', or 'ASYM'")
    
    d = loss1 - loss2
    return d

def bartlett_kernel(j, H):
    return 1 - (j / H)

def long_run_variance(d, H):
    T = len(d)
    gamma_0 = np.var(d, ddof=1)
    gamma_sum = 0
    for j in range(1, H+1):
        weight = bartlett_kernel(j, H)
        cov = np.cov(d[:-j], d[j:])[0, 1]
        gamma_sum += weight * cov
    LRV = gamma_0 + 2 * gamma_sum
    return LRV

def dm_statistic(mean_d, LRV, T):
    S = mean_d / np.sqrt(LRV / T)
    return S

def dm_test_manual(actual, forecast1, forecast2, h=1, crit="MSE", lambda_=0.1):
    d = calculate_loss_differential(actual, forecast1, forecast2, crit, lambda_)
    mean_d, T = np.mean(d), len(d)
    H = int(4 * (T / 100) ** (2/9))  # Optimal bandwidth parameter
    LRV = long_run_variance(d, H)
    S = dm_statistic(mean_d, LRV, T)
    p_value = 2 * (1 - stats.t.cdf(np.abs(S), df=T-1))
    return {"DM": S, "p_value": p_value}



# List of stocks to analyze
stocks = ["AAPL", "SPY", "IBM", "PG", "WMT", "DIS", "JPM"]

# Models and horizons
models = ["realised_garch_horizon_", "realised_har_garch_horizon_"]
horizons = [1, 5, 10, 20]

# Values of lambda to test
lambdas = [0.1, 0.5, 1, 2]

# Placeholder for results
results_df = pd.DataFrame()

# Loop through each stock and each model to calculate metrics
for stock in stocks:
    # Load actual data from CSV
    actual_data_path = os.path.join(test_data_dir, f"{stock}_test_data.csv")
    actual_data = pd.read_csv(actual_data_path)
    actual = actual_data['log_x_adj']
    
    for horizon in horizons:
        for i in range(len(models) - 1):
            for j in range(i + 1, len(models)):
                model1_name = f"{models[i]}{horizon}"
                model2_name = f"{models[j]}{horizon}"
                
                forecast_file1 = os.path.join(forecast_dir, f"{stock}.csv", f"{model1_name}.csv")
                forecast_file2 = os.path.join(forecast_dir, f"{stock}.csv", f"{model2_name}.csv")
                
                if os.path.exists(forecast_file1) and os.path.exists(forecast_file2):
                    forecast_data1 = pd.read_csv(forecast_file1)
                    forecast_data2 = pd.read_csv(forecast_file2)
                    
                    forecast1 = forecast_data1['forecast_data']
                    forecast2 = forecast_data2['forecast_data']
                    
                    actual_adjusted = actual[:len(actual) - (horizon - 1)]
                    
                    # Ensure the forecast length matches the adjusted actual length
                    forecast1 = forecast1[:len(actual_adjusted)]
                    forecast2 = forecast2[:len(actual_adjusted)]
                    


    Stock  Horizon                     Model1                         Model2  \
0    AAPL        1   realised_garch_horizon_1   realised_har_garch_horizon_1   
1    AAPL        1   realised_garch_horizon_1   realised_har_garch_horizon_1   
2    AAPL        1   realised_garch_horizon_1   realised_har_garch_horizon_1   
3    AAPL        1   realised_garch_horizon_1   realised_har_garch_horizon_1   
4    AAPL        5   realised_garch_horizon_5   realised_har_garch_horizon_5   
..    ...      ...                        ...                            ...   
107   JPM       10  realised_garch_horizon_10  realised_har_garch_horizon_10   
108   JPM       20  realised_garch_horizon_20  realised_har_garch_horizon_20   
109   JPM       20  realised_garch_horizon_20  realised_har_garch_horizon_20   
110   JPM       20  realised_garch_horizon_20  realised_har_garch_horizon_20   
111   JPM       20  realised_garch_horizon_20  realised_har_garch_horizon_20   

     Lambda  DM_test_statistic_ASYM  DM

In [40]:
 # Perform DM test manually for each lambda
for lambda_ in lambdas:
    dm_test_result_asym = dm_test_manual(actual_adjusted, forecast1, forecast2, h=horizon, crit="ASYM", lambda_=lambda_)

    # Create a DataFrame for the current result
    current_result = pd.DataFrame([{
        "Stock": stock,
        "Horizon": horizon,
        "Model1": model1_name,
        "Model2": model2_name,
        "Lambda": lambda_,
        "DM_test_statistic_ASYM": dm_test_result_asym["DM"],
        "DM_p_value_ASYM": dm_test_result_asym["p_value"]
    }])

    # Concatenate with the results_df DataFrame
    results_df = pd.concat([results_df, current_result], ignore_index=True)

    results_df.to_csv('/Users/raphaelravinet/Code/BSE/Thesis/Results/DM_TEST/dm_test_results_asymetry.csv')

In [45]:
import os
import numpy as np
import pandas as pd
from scipy import stats

def calculate_loss_differential(actual, forecast1, forecast2, crit="MSE"):
    if crit == "MSE":
        loss1 = (actual - forecast1) ** 2
        loss2 = (actual - forecast2) ** 2
    elif crit == "MAD":
        loss1 = np.abs(actual - forecast1)
        loss2 = np.abs(actual - forecast2)
    else:
        raise ValueError("Criterion must be either 'MSE' or 'MAD'")
    
    d = loss1 - loss2
    return d

def bartlett_kernel(j, H):
    return 1 - (j / H)

def long_run_variance(d, H):
    T = len(d)
    gamma_0 = np.var(d, ddof=1)
    gamma_sum = 0
    for j in range(1, H+1):
        weight = bartlett_kernel(j, H)
        cov = np.cov(d[:-j], d[j:])[0, 1]
        gamma_sum += weight * cov
    LRV = gamma_0 + 2 * gamma_sum
    return LRV

def dm_statistic(mean_d, LRV, T):
    S = mean_d / np.sqrt(LRV / T)
    return S

def dm_test_manual(actual, forecast1, forecast2, h=1, crit="MSE"):
    d = calculate_loss_differential(actual, forecast1, forecast2, crit)
    mean_d, T = np.mean(d), len(d)
    H = int(4 * (T / 100) ** (2/9))  # Optimal bandwidth parameter
    LRV = long_run_variance(d, H)
    S = dm_statistic(mean_d, LRV, T)
    p_value = 2 * (1 - stats.t.cdf(np.abs(S), df=T-1))
    return {"DM": S, "p_value": p_value}

# Directory paths
forecast_dir = "/Users/raphaelravinet/Code/BSE/Thesis/Results/Forecast_results_final"
test_data_dir = "/Users/raphaelravinet/Code/BSE/Thesis/Results/DM_TEST"

# List of stocks to analyze
stocks = ["AAPL", "SPY", "IBM", "PG", "WMT", "DIS", "JPM"]

# Models and horizons
models = ["realised_garch_horizon_", "realised_har_garch_horizon_"]
horizons = [1, 5, 10, 20]

# Placeholder for results
results_df = pd.DataFrame()

# Loop through each stock and each model to calculate metrics
for stock in stocks:
    # Load actual data from CSV
    actual_data_path = os.path.join(test_data_dir, f"{stock}_test_data.csv")
    actual_data = pd.read_csv(actual_data_path)
    actual = actual_data['log_x_adj']
    
    for horizon in horizons:
        for i in range(len(models) - 1):
            for j in range(i + 1, len(models)):
                model1_name = f"{models[i]}{horizon}"
                model2_name = f"{models[j]}{horizon}"
                
                forecast_file1 = os.path.join(forecast_dir, f"{stock}.csv", f"{model1_name}.csv")
                forecast_file2 = os.path.join(forecast_dir, f"{stock}.csv", f"{model2_name}.csv")
                
                if os.path.exists(forecast_file1) and os.path.exists(forecast_file2):
                    forecast_data1 = pd.read_csv(forecast_file1)
                    forecast_data2 = pd.read_csv(forecast_file2)
                    
                    forecast1 = forecast_data1['forecast_data']
                    forecast2 = forecast_data2['forecast_data']
                    
                    actual_adjusted = actual[:len(actual) - (horizon - 1)]
                    
                    # Ensure the forecast length matches the adjusted actual length
                    forecast1 = forecast1[:len(actual_adjusted)]
                    forecast2 = forecast2[:len(actual_adjusted)]
                    
                    # Perform DM test manually with MSE
                    dm_test_result_mse = dm_test_manual(actual_adjusted, forecast1, forecast2, h=horizon, crit="MSE")
                    
                    # Create a DataFrame for the current result
                    current_result = pd.DataFrame([{
                        "Stock": stock,
                        "Horizon": horizon,
                        "Model1": model1_name,
                        "Model2": model2_name,
                        "DM_test_statistic_MSE": dm_test_result_mse["DM"],
                        "DM_p_value_MSE": dm_test_result_mse["p_value"]
                    }])
                    
                    # Concatenate with the results_df DataFrame
                    results_df = pd.concat([results_df, current_result], ignore_index=True)



In [46]:
results_df

,Stock,Horizon,Model1,Model2,DM_test_statistic_MSE,DM_p_value_MSE
0,AAPL,1,realised_garch_horizon_1,realised_har_garch_horizon_1,7.465740,1.523226e-13
1,AAPL,5,realised_garch_horizon_5,realised_har_garch_horizon_5,21.610257,0.000000e+00
2,AAPL,10,realised_garch_horizon_10,realised_har_garch_horizon_10,27.080255,0.000000e+00
3,AAPL,20,realised_garch_horizon_20,realised_har_garch_horizon_20,27.138034,0.000000e+00
4,SPY,1,realised_garch_horizon_1,realised_har_garch_horizon_1,5.181915,2.548197e-07
5,SPY,5,realised_garch_horizon_5,realised_har_garch_horizon_5,9.891687,0.000000e+00
6,SPY,10,realised_garch_horizon_10,realised_har_garch_horizon_10,13.165348,0.000000e+00
7,SPY,20,realised_garch_horizon_20,realised_har_garch_horizon_20,11.619116,0.000000e+00
8,IBM,1,realised_garch_horizon_1,realised_har_garch_horizon_1,-1.034275,3.012037e-01
9,IBM,5,realised_garch_horizon_5,realised_har_garch_horizon_5,2.403428,1.638457e-02


In [62]:
import pandas as pd
import os
from scipy.stats import skew, kurtosis

# Define the directory containing the CSV files
directory = '/Users/raphaelravinet/Code/BSE/Thesis/final_datasets copy 2'

# Define the metrics to compute
metrics = ['Obs', 'Mean', 'Std', 'Median', 'Skew', 'Kurt', 'Q1', 'Q3']

# Function to compute statistics
def compute_stats(data, column, metrics):
    stats = {}
    if 'Obs' in metrics:
        stats['Obs'] = len(data)
    if 'Mean' in metrics:
        stats['Mean'] = data[column].mean()
    if 'Std' in metrics:
        stats['Std'] = data[column].std()
    if 'Median' in metrics:
        stats['Median'] = data[column].median()
    if 'Skew' in metrics:
        stats['Skew'] = skew(data[column])
    if 'Kurt' in metrics:
        stats['Kurt'] = kurtosis(data[column], fisher=False)
    if 'Q1' in metrics:
        stats['Q1'] = data[column].quantile(0.25)
    if 'Q3' in metrics:
        stats['Q3'] = data[column].quantile(0.75)
    return stats

# Initialize a list to hold all results
all_results = []

# Process each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        data = pd.read_csv(filepath)
        
        # Multiply 'returns' and 'RM' by 100
        if 'returns' in data.columns:
            data['returns'] = data['returns'] * 100
        if 'RM' in data.columns:
            data['RM'] = data['RM'] * 100
        
        # Compute statistics for 'returns' and 'RM'
        returns_stats = compute_stats(data, 'returns', metrics) if 'returns' in data.columns else {}
        rm_stats = compute_stats(data, 'RM', metrics) if 'RM' in data.columns else {}
        
        # Combine statistics into a DataFrame for LaTeX
        stats_df = pd.DataFrame({
            'Metric': metrics,
            'Returns': [returns_stats.get(metric, 'N/A') for metric in metrics],
            'RM': [rm_stats.get(metric, 'N/A') for metric in metrics]
        })
        
        # Append to all results
        all_results.append((filename, stats_df))

# Generate LaTeX tables
latex_tables = []
for filename, stats_df in all_results:
    latex_table = stats_df.to_latex(multicolumn=True, multicolumn_format='c', index=False, float_format="%.2f", 
                                    column_format="l" + "r" * (len(stats_df.columns) - 1), 
                                    caption=f"Summary statistics for {filename}")
    latex_tables.append(latex_table)

# Save all LaTeX tables to a file
with open('summary_statistics.tex', 'w') as f:
    for latex_table in latex_tables:
        f.write(latex_table)
        f.write('\n\n')
